<a href="https://colab.research.google.com/github/Ishwari208/network-intrusion-detection-ml/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [112]:
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt



--2026-02-13 14:57:17--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19109424 (18M) [text/plain]
Saving to: ‘KDDTrain+.txt.14’

KDDTrain+.txt.14    100%[===================>]  18.22M  --.-KB/s    in 0.1s    

2026-02-13 14:57:17 (132 MB/s) - ‘KDDTrain+.txt.14’ saved [19109424/19109424]



In [113]:
import pandas as pd

data = pd.read_csv("KDDTrain+.txt", header=None)

print("Dataset Shape:", data.shape)
data.head()


Dataset Shape: (125973, 43)


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [114]:
data.iloc[:, -2].value_counts().head(10)


,count
41,
normal,67343
neptune,41214
satan,3633
ipsweep,3599
portsweep,2931
smurf,2646
nmap,1493
back,956
teardrop,892


In [115]:
# Create binary label
data['label'] = data.iloc[:, -2].apply(lambda x: 0 if x == 'normal' else 1)

data['label'].value_counts()


,count
label,
0,67343
1,58630


In [116]:
# Features (remove original label column and difficulty column)
X = data.iloc[:, :-2]

# Target
y = data['label']

print("Feature shape:", X.shape)
print("Target shape:", y.shape)


Feature shape: (125973, 42)
Target shape: (125973,)


In [117]:
# Drop original attack label column (index 41)
# Drop difficulty column (index 42)
# Drop our new binary label column from features

X = data.drop(columns=[41, 42, 'label'])

y = data['label']

print("Feature shape:", X.shape)
print("Target shape:", y.shape)


Feature shape: (125973, 41)
Target shape: (125973,)


In [118]:
X.dtypes.value_counts()


,count
int64,23
float64,15
object,3


In [119]:
X.select_dtypes(include='object').columns


Index([1, 2, 3], dtype='object')

In [120]:
# One-hot encode categorical columns
X_encoded = pd.get_dummies(X, columns=[1, 2, 3])

print("Encoded feature shape:", X_encoded.shape)


Encoded feature shape: (125973, 122)


In [121]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print("Training set:", X_train.shape)
print("Testing set:", X_test.shape)


Training set: (100778, 122)
Testing set: (25195, 122)


In [122]:
# Convert all column names to string
X_encoded.columns = X_encoded.columns.astype(str)

# Re-split again (important after modification)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)


In [123]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train, y_train)

y_pred_lr = model_lr.predict(X_test)

print("Model trained successfully.")


Model trained successfully.


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
